In [5]:
import numpy as np
import qutip as qt
import matplotlib.pyplot as plt
import random
import itertools
import time
from joblib import Parallel, delayed

In [6]:
## general functions
def create_pauli_basis(n):
    # return list with all n-qubit Paulis
    s = [qt.qeye(2), qt.sigmax(), qt.sigmay(), qt.sigmaz()]
    p = s
    for i in range(n-1):
        p = [qt.tensor(pi, si) for pi in p for si in s]
    r = [np.array(pi) for pi in p]
    return r

def dm_to_bvector(a, basis, dim):
    # convert density matrix to vector in pauli basis (if basis = pauli basis)
    return 1 / dim * np.real(np.array([np.trace(np.dot(np.array(a), bi)) for bi in basis]))

def ket_to_bvector(a, basis, dim):
    # convert ket to vector in pauli basis (if basis = pauli basis)
    aa = np.array(a).T
    return 1 / dim * np.real(np.array([np.trace(np.dot(np.dot(np.conj(aa.T), bi), aa)) for bi in basis]))

def bvector_to_dm(v, basis):
    # convert vector in pauli basis to density matrix (if basis = pauli basis)
    return qt.Qobj(np.sum(np.array([v[i] * basis[i] for i in range(len(v))]), axis= 0))

In [7]:
## functions for estimation experiment

def sample_ginibre_ensemble(n, p, dim_n, dim_k=None):
    # draw n states from the ginibre distribution (unbiased)
    x_0 = np.zeros((n, dim_n**2))
    w_0 = np.ones(n)/n
    for i in range(n):
        dm = qt.rand_dm_ginibre(N=dim_n, rank=dim_k)
        x_0[i] = dm_to_bvector(dm, p, dim_n) # calculate pauli representation
    return x_0, w_0

def POVM_randbasis(M, p, dim):
    # returns dim orthogonal directions, sampled according to the haar measure
    o = np.zeros((M, dim, dim**2))
    for m in range(M):
        u = qt.rand_unitary_haar(dim)
        o[m] = np.array([ket_to_bvector(u[i], p, dim) for i in range(dim)])
    return o

def POVM_paulibasis(M, p, dim):
    # returns dim orthogonal directions, sampled according from the Pauli basis
    n_q = int(np.log2(dim))
    o = np.zeros((M, dim, dim**2))
    b = np.random.randint(1, 4, size= (M, n_q))
    ind = np.array([4**i for i in range(n_q)][::-1])
    signs = np.array([list(i) for i in itertools.product([-1, 1], repeat= n_q)])
    comb = np.array([list(i) for i in itertools.product([0, 1], repeat= n_q)]) 
    for m in range(M):
        for ids, s in enumerate(signs):
            for c in comb:
                o[m][ids][np.sum(ind*c*b[m])] = (-1)**(len(np.where(c*s==-1)[0])%2)
    return o / dim

def prob_projectivemeas(oi, rho):
    # outcome probabilities of projective measurements specified in o, when measuring rho
    dim = np.sqrt(len(rho))
    prob = dim * np.array([np.sum(oo * rho) for oo in oi])
    if abs(np.sum(prob)-1) > 0.1: print(np.sum(prob))
    return prob
                                          
def experiment(o, rho):
    # measure rho in basis specified in POVM elements o
    dim = len(o[0])
    x = np.zeros(len(o))
    for ido, oi in enumerate(o):
        prob = prob_projectivemeas(oi, rho)
        x[ido] = np.random.choice(np.arange(dim), p= prob)
    return x
                                          
def likelihood(r, xi, oi):
    # calculate likelihood of measurement outcomes x for states in an array r 
    lh = np.array([np.sum(oi[int(xi)] * ri) for ri in r]) # proportional to probability (* dim is missing)
    return lh

def bayes_update(r, w, x, o, n_active, threshold):
    # update weights according to likelihood and normalize    
    start = time.time()
    w_temp = w
    for i in range(len(x)):
        w_new = np.zeros(len(w_temp)) # needed such that weights below the threshold are 0
        w_new[n_active] = w_temp[n_active] * likelihood(r[n_active], x[i], o[i])
        w_new[n_active] = np.divide(w_new[n_active], np.sum(w_new[n_active]))
        w_temp = w_new
        n_active = n_active[np.where(w_new[n_active] > threshold)]
        end = time.time()
    return w_new, end-start
                                  
def pointestimate(x, w):
    # return point estimate of rho
    return np.average(x, axis=0, weights= w)

def fidelity(a, b, p):
    # compute fidelity from density matrices in Pauli representation
    return qt.metrics.fidelity(bvector_to_dm(a, p), bvector_to_dm(b, p))**2

In [28]:
# ESTIMATION

# system
n_q = 2 # number of Qubits - fixed in this implementation
dim = 2**n_q # dimension of Hilbert space
p = create_pauli_basis(n_q) # create Pauli basis

# measurements
M = 1000 # number of measurements
O = POVM_paulibasis(M, p, dim)
#O = POVM_randbasis(M, p, dim)

# ensemble
L = 1000 # number of sampling points
r, w0 = sample_ginibre_ensemble(L, p, dim, dim)
rho = r[0] # Ideal state

# cutoff for weights
threshold = 1 / (L**2)
n_active0 = np.arange(L)

In [29]:
# COMPUTATION
def function(rho, r, w0, O, p, n_active0, threshold):
    start = time.time()
    x = experiment(O, rho)
    w, _ = bayes_update(r, w0, x, O, n_active0, threshold)
    rho_est = pointestimate(r, w)
    fid = np.round(fidelity(rho, rho_est, p), decimals= 7)
    n_ess = np.round(1 / np.sum(w**2), decimals= 4)
    w_max = np.round(np.max(w), decimals= 4)
    duration = np.round(time.time() - start, decimals= 3)
    return fid, n_ess, w_max, duration

In [34]:
t_0 = time.time()
out = Parallel(n_jobs=-2)(delayed(function)(rho, r, w0, O, p, n_active0, threshold) for rep in range(20))
duration = time.time() - t_0

In [35]:
print(np.sum(np.array(out)[:,3]))
print(duration)

16.798000000000002
2.706873893737793


In [ ]:
# Simulate experiment
x = experiment(O, rho)

In [ ]:
# Update the weights of the ensembles
w, dt = bayes_update(r, w0, x, O, n_active0, threshold)

In [ ]:
# Estimation
rho_est = pointestimate(r, w)
print(fidelity(rho, rho_est, p))
print(dt)